# Law, Bias, and Algorithms
## Algorithmic fairness (1/2)

In 2016, ProPublica published a [now-famous article](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) analyzing and criticizing the lack of fairness in a risk assessment tool used nationwide called COMPAS. Here, we will take a look at a cleaned-up version of the COMPAS data that ProPublica used, and try to better understand algorithmic fairness by investigating the claims ProPublica made, along with the [counterclaims](https://www.propublica.org/article/technical-response-to-northpointe) made by Northpointe (now re-branded as [Equivant](https://www.equivant.com/)).

While Northpointe notes that their algorithm does not use race information and that their model is calibrated across racial groups, ProPublica points out that the COMPAS scores differ in false positive rates across racial groups (violating classification parity). In this notebook, we will examine some of their claims by building and evaluating our own risk assessment tool.

In [0]:
# Some initial setup
options(digits = 3)
library(tidyverse)

theme_set(theme_bw())

# Because huge plots are ugly
options(repr.plot.width = 6, repr.plot.height = 4)

# Read the data
compas_df <- read_rds("../data/compas.rds")

## COMPAS data

A cleaned version of the COMPAS data is loaded as `compas_df`, with the following columns

* `id`: unique identifiers for each case
* `sex`, `dob`, `age`, `race`: demographic information for each defendant
* `recid_score`, `violence_score`: COMPAS scores assessing risk that a defendant will recidivate (`violence_score` for violent crimes) within two years of release (higher scores correspond to higher risk)
* `priors_count`: number of prior arrests
* `is_recid`, `is_violent_recid`: Indicator variable that is `1` if the defendant was arrested for a new (violent) crime within two years of release, and `0` otherwise.

In [0]:
head(compas_df)

### Exercise 1: Build a risk assessment model for recidivism

We start by building our own risk assessment tool using only prior arrests (`priors_count`) and age (`age`) to predict whether a defendant will recidivate within two years of release (`is_recid`).
First, fit a model to estimate the probability of this outcome for each defendant. 
We will call this model `recid_model`.

In [0]:
# Build a logistic regression model estimating recidivism probability

recid_model <- 
# WRITE CODE HERE


Given an estimate of risk (on the probability scale), we can compute a binary prediction of whether a defendant will recidivate or not by setting a decision threshold.
Once a threshold is determined, defendants with estimated risk higher or equal to the threshold are predicted to recidivate.
We can then compare the binary prediction to the observed outcome (`is_recid`) to compute the accuracy of our risk assessment tool at the specified threshold.
We can similarly compute accuracy for COMPAS scores (`recid_score`) with some threshold.

Calculate the accuracy for the `recid_model` that you fit above at a $50$% threshold, and for the COMPAS recidivism scores at a threshold of $4$ (which corresponds to approximately $50$%).

In [0]:
# WRITE CODE HERE


### Exercise 2: Calibration

We next examine how well our model is calibrated across different race groups (i.e., for people who receive similar risk scores, what is the actual rate of recidivism across each race group).

To do so, we calculate two quantities:
* `predicted_risk_score`: a discretized (rounded) version of our predicted risk from Exercise 1, similar to the COMPAS risk score;
* `recidivism_rate`: the actual recidivism rate of people within a specific discretized risk score bucket.

Calculate the above two quantities for both race groups in our dataset by creating a data frame called `calibration_by_race` containing three columns: `race`, `predicted_risk_score` (rounded to the nearest 10 percentage points), `recidivism_rate`.

_Hint_: `round(x)` will round each element of `x` to the nearest integer.

In [0]:
# Calculate discretized risk score

calibration_by_race <- compas_df %>%
# WRITE CODE HERE

# Put the recidivism rates of different races side by side
calibration_by_race %>%
    spread(race, recidivism_rate)

We can also visualize model calibration by plotting the risk score bins with their corresponding emprical recidivism rate:

In [0]:
# Calibration plot
ggplot(calibration_by_race, 
       aes(x = predicted_risk_score, y = recidivism_rate, color = race)) +
    geom_line() + 
    scale_y_continuous(labels = scales::percent_format(), limits = c(0, 1))+
    scale_x_continuous(breaks = seq(0, 10, 2), limits = c(1, 10))+
    labs(x = "\nDiscretized risk score",
         y = "Recidivism rate\n")

The plot above suggets our model is well calibrated for black and white defendants.
In other words, black and white defendants who receive similar risk scores have similar risk of recidivating.

### Exercise 3: Cost-benefit analysis

Detaining an individual incurs substantial financial and social costs, 
yet releasing a defendant can harm public safety.
One strategy is to detain individuals with high recidivism risk while releasing those with low recidivism risk.
In order to make such a detention policy based on our model, we need to decide on a detention threshold.

To inform that decision, we'll attempt to estimate, for any given detention threshold,
the proportion of crimes that would be prevented and at what cost.
For the former, we'll compute the proportion of defendants who ultimately commit another crime that are detained under the policy; for the latter, we'll compute the proportion of all defendants that are detained.

For this exercise, complete the following function which, given a data frame and a threshold, returns a new data frame including two columns: 
* `prop_crime_prevented`: the proportion of crime that is prevented, and
* `prop_detained`: the proportion of the population that is detained

We will assume that the data frame passed to the function contains at least the following columns:
* `risk`: risk score that is used for the policy
* `is_recid`: outcome indicating whether a defendant recidiviated

In [0]:
calc_cost_benefit <- function(d, threshold){
    d %>%
# WRITE CODE HERE
}

calc_cost_benefit(compas_df, threshold = 0.5)

Explore the cost-benefit characteristics of different policies that use different thresholds. 

We can also plot this cost-benefit tradeoff for all possible detention thresholds using the following code:

In [0]:
cost_benefit_df <- map_dfr(unique(compas_df$risk), 
                           ~ calc_cost_benefit(compas_df, .) %>%
                               mutate(threshold = .x))

# Plot proportion detained vs. proportion crime prevented
options(repr.plot.width = 4, repr.plot.height = 4)
ggplot(cost_benefit_df, aes(x=prop_detained, y=prop_crime_prevented)) +
    geom_line()+
    scale_y_continuous(labels = scales::percent_format(), limits=c(0, 1))+
    scale_x_continuous(labels = scales::percent_format(), limits=c(0, 1))+
    labs(x = "\nProportion detained",
         y = "Proportion crime prevented\n")

### Exercise 4: Disparities in detention

By conducting the above cost-benefit analysis,
we can decide on a detention threshold, say $0.5$, that we believe can maximize our benefit (crime prevention) at a acceptable cost of detaining some of the riskest defendants according to our model.

Now, we would like to know whether such a facially neutral policy, where we detain everyone at the same threshold, will introduce disparities across defendants of different races.

To do so, let's compute the dentention rate for each race group using a detention threshold of 0.5.

In [0]:
# Calculate detention and recidivism rate by race

# WRITE CODE HERE


Despite the same detention threshold being used, we still observe a difference in dentention rates.

To further investigate, we can plot the distribution of risk for each of the two groups:

In [0]:
# Plot the risk distribution
options(repr.plot.width = 7, repr.plot.height = 3.5)
ggplot(compas_df, aes(x = risk, fill = race)) +
    geom_density(alpha = 0.5, color = NA) +
    scale_x_continuous("Estimated risk", labels = scales::percent_format(), expand = c(0, 0)) +
    scale_y_continuous(element_blank(), expand = c(0, 0)) +
    theme(axis.ticks.y = element_blank(),
          axis.text.y = element_blank())

Assuming that our model provides an accurate assessment of risk, we see that white and black defendents have different risk distributions.
When we are making a policy decision, we are often concerned with what happens at the margin 
(e.g., whether the same standard is applied to all individuals).
However, popular error metrics assess behavior away from the margin, hence they are called infra-marginal statistics.
This general phenomenon is known as the problem of _infra-marginality_ (Ayres, 2002; Simoiu et al., 2017; Corbett-Davies and Goel, 2018).

### Exercise 5: A "fair" policy by equalizing detention rates?

After observing differences in the detention rates above, one natural thought might be to find a policy which equalizes detention rates across race groups.

In this exercise, play around with different thresholds for black and white defendants to achieve a detention rate of approximately 50% for each group.

In [0]:
black_threshold = 0.5
white_threshold = 0.5

# WRITE CODE HERE

# Calculate detention and recidivism rate by race
compas_df %>%
    mutate(detained = risk > if_else(race == "Caucasian", white_threshold, black_threshold)) %>%
    group_by(race) %>%
    summarize(
        detention_rate = mean(detained),
    )


What are the threshold values that you find? Which policy would you consider "fair", and why?